In [0]:
import sys #librerias para movernos entre carpetas
import os #librerias para movernos entre carpetas
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator #Nos ayudara a preprocesar las imagenes
from tensorflow.python.keras import optimizers #optimizador con el cual vamos a entrenar nuestro algoritmo
from tensorflow.python.keras.models import Sequential #Libreria que nos permite hacer redes neuronales secuenciales
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D #Son las capas donde haremos nuestras convoluciones y nuestro maxpooling
 

In [2]:
from google.colab import drive
drive.mount("./drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./drive


In [0]:
!cp drive/"My Drive"/carpetas/data.zip .
!unzip data.zip;

In [0]:
!ls drive/"My Drive"

In [0]:
!rm data.zip

In [0]:
data_entrenamiento = './data/entrenamiento'
data_validacion = './data/validacion'

In [0]:
"""
Parameters
"""
epocas=20 #Numero de veces que vamos a iterar nuestras imagenes en nuestro modelo
longitud, altura = 150, 150 # Tamaño en que se va a procesar nuestras imagenes
batch_size = 32 #Numero de imagenes que se va a mandar por cada paso
pasos = 1000 # Numero de veces que se va a procesar la informacion en cada una de las epocas
validation_steps = 300 #al final de cada epoca se correra para ver que tan bien esta aprendiendo nuestro algoritmo
filtrosConv1 = 32 #
filtrosConv2 = 64 # despues de cada convolucion cambia la profundidad de la imagen
tamano_filtro1 = (3, 3)  #
tamano_filtro2 = (2, 2) #tamaño del filtro
tamano_pool = (2, 2) #tamaño del filtro del maxpooling
clases = 2 
lr = 0.001 #learningrate: que tan grande son los ajustes que hara nuestra red para acercarse a una solución optima

In [0]:
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255, #cada uno de nuestros pixeles tienen un rango de 0 a 255 y lo estamos cambiando de 0 a 1
    shear_range=0.2, #generar imagenes, pero las va a inclinar
    zoom_range=0.2, #a algunas les va a hacer zoom
    horizontal_flip=True) #va a invertir algunas imagenes

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [12]:
entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')


Found 980 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [0]:
cnn = Sequential() #vamos a generar una red secuancial, es decir varias capas apiladas
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten()) #aplanamos la imagen
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))#apaga la mitad de las neuronas por cada paso
cnn.add(Dense(clases, activation='softmax'))

cnn.compile(  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],)

In [0]:
cnn.fit_generator(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
1000/1000 [==============================] - 246s 246ms/step - loss: 0.5879 - accuracy: 0.7435 - val_loss: 0.7477 - val_accuracy: 0.6927
Epoch 2/20
1000/1000 [==============================] - 242s 242ms/step - loss: 0.2024 - accuracy: 0.9191 - val_loss: 1.2244 - val_accuracy: 0.7215
Epoch 3/20
1000/1000 [==============================] - 240s 240ms/step - loss: 0.1185 - accuracy: 0.9572 - val_loss: 1.1627 - val_accuracy: 0.7167
Epoch 4/20
1000/1000 [==============================] - 239s 239ms/step - loss: 0.0878 - accuracy: 0.9695 - val_loss: 1.3594 - val_accuracy: 0.7112
Epoch 5/20
1000/1000 [==============================] - 238s 238ms/step - loss: 0.0744 - accuracy: 0.9743 - val_loss: 1.4750 - val_accuracy: 0.7207
Epoch 6/20
1000/1000 [==============================] - 242s 242ms/step - loss: 0.0657 - accuracy: 0.9781 - val_loss: 1.5857 - val_accuracy: 0.7163
Epoch 7/20
1000/1000 [==============

In [0]:
target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')

In [0]:
print(entrenamiento_generador.class_indices)

{'gato': 0, 'perro': 1}


In [16]:
!cp drive/"My Drive"/modelo.zip .
!unzip modelo.zip;

Archive:  modelo.zip
  inflating: modelo/modelo.h5        
  inflating: modelo/pesos.h5         


In [0]:
!rm modelo.zip


In [19]:
import numpy as np
import tensorflow as tf 
from keras.preprocessing.image import load_img, img_to_array
#from keras.models import load_model

Using TensorFlow backend.


In [20]:
longitud, altura = 150, 150
modelo = './modelo/modelo.h5'
pesos_modelo = './modelo/pesos.h5'
cnn = tf.keras.models.load_model(modelo)
cnn.load_weights(pesos_modelo)


In [0]:
def predict(file):
  x = load_img(file, target_size=(longitud, altura))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = cnn.predict(x) # [[1,0]]
  result = array[0] #[1,0]
  answer = np.argmax(result)
  dic ={0:'gato', 1:'perro'}
  print(dic[answer])
  return answer

In [26]:
predict('./data/perro.jpg')

perro


1

In [23]:
!ls data

entrenamiento  gato2.jpg  gato3.jpg  perro1.jpg  perro.jpg  validacion
